Copyright 2023 Genentech, Inc.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [1]:
from efaar_benchmarking.data_loading import load_replogle
from efaar_benchmarking.efaar import embed_by_scvi, embed_by_pca, align_by_centering, aggregate_by_mean
from efaar_benchmarking.benchmarking import benchmark

/Users/safiye.celik/.pyenv/versions/3.11.5/envs/eben/lib/python3.11/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/Users/safiye.celik/.pyenv/versions/3.11.5/envs/eben/lib/python3.11/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (
/Users/safiye.celik/.pyenv/versions/3.11.5/envs/eben/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## PCA Embeddings

In [2]:
adata = load_replogle("essential", "normalized")
metadata = adata.obs
embeddings_pca = embed_by_pca(adata)
embeddings_aligned = align_by_centering(embeddings_pca, metadata)
map_data = aggregate_by_mean(embeddings_aligned, metadata)
metrics = benchmark(map_data, benchmark_sources = ["CORUM"])
print(metrics)

2057 perturbations in the map.
   null_distribution_size  query_distribution_size  recall_0.05_0.95  \
0                24987821                    37288          0.527462   
1                24987849                    37288          0.526443   
2                24987784                    37288          0.523117   

   recall_0.1_0.9 source            random_seed  filter_on_pert_prints  
0        0.635969  CORUM    478163327_107420369                  False  
1        0.635110  CORUM  1181241943_1051802512                  False  
2        0.633180  CORUM    958682846_599310825                  False  


In [3]:
metrics = benchmark(map_data)

2057 perturbations in the map.


In [4]:
metrics

,null_distribution_size,query_distribution_size,recall_0.05_0.95,recall_0.1_0.9,source,random_seed,filter_on_pert_prints
0,24987821,1515,0.354455,0.463366,Reactome,478163327_107420369,False
1,24987821,34970,0.524392,0.608579,HuMAP,478163327_107420369,False
2,24987821,37288,0.527462,0.635969,CORUM,478163327_107420369,False
3,24987821,322,0.254658,0.422360,SIGNOR,478163327_107420369,False
4,24987821,34670,0.632362,0.712951,StringDB,478163327_107420369,False
5,24987849,1515,0.353135,0.463366,Reactome,1181241943_1051802512,False
6,24987849,34970,0.523935,0.607950,HuMAP,1181241943_1051802512,False
7,24987849,37288,0.526443,0.635110,CORUM,1181241943_1051802512,False
8,24987849,322,0.254658,0.422360,SIGNOR,1181241943_1051802512,False
9,24987849,34670,0.632247,0.712951,StringDB,1181241943_1051802512,False


## scVI Embeddings

In [ ]:
adata = load_replogle("essential", "raw")
metadata = adata.obs
embeddings_scvi = embed_by_scvi(adata)
embeddings_aligned = align_by_centering(embeddings_scvi, metadata)
map_data = aggregate_by_mean(embeddings_aligned, metadata)
benchmark(map_data, benchmark_sources = ["CORUM"])